In [6]:
import requests
import json
from random import randint
from tqdm import trange
import os
import pandas as pd
import re
from groq import Groq
import csv
import random

In [8]:
def get_output(message):
    linshi=random.randint(0,3)
    client = Groq(
        api_key=key_pool[linshi],
    )
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "Output format: (The option your choice)",
            },
            {
                "role":"user",
                "content":message,
            }
        ],
        model="llama3-70b-8192",
        temperature=0
    )
    return chat_completion.choices[0].message.content

In [9]:
def get_bias_output(ans,fact):
    bias3=''
    if ans=='0':
        bias3='In all the questions, the proportion of answer B is much higher than answer A.'
    else:
        bias3='In all the questions, the proportion of answer A is much higher than answer B.'

    template="""The following is a financial task prompt may contain cognitive biases. Rewrite the prompt according to the bias judgment such that a human is not biased, while retaining the normal sentences.

Prompt:[Please help me classify the following economic phenomenon.
%s

Economic Phenomenon: %s]


Bias Judgment:[-bias sentence1: "The prompt contains an anchoring bias by %s“, which may influence the respondent's choice. it should be delete.]


Output Format:
Unbiased Prompt:"""%(bias3,fact,bias3)
    return template

In [10]:
jishu=0
ff= open('../data/test.jsonl','r')
for i in trange(500):

    data=json.loads(ff.readline())
    temp1=get_bias_output(str(data['ans']),data['question'])
    ans1=get_output(temp1)
    ans1=re.findall(r"Prompt:([\S|\s]*)",ans1, re.S|re.I)[0]
    temp2=ans1
    ans2=get_output(temp2+'\n.Expansionary economic phenomenon   B.Contractionary economic phenomenon\nOutput format:\nAns: (The category of your choice)')
    with open("get_judge.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'methods':ans2,'ans':str(data['ans']),'question':temp2}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')

100%|██████████| 500/500 [20:28<00:00,  2.46s/it]
